Student Information

Name: 劉榮萱 

Student ID: 106003852

GitHub ID: ll890

Kaggle name: Linda Liou

In [1]:
import nltk
import pandas as pd
import json

In [2]:
# read files
df_di = pd.read_csv('data_identification.csv')
df_em = pd.read_csv('emotion.csv')
data = [json.loads(line) for line in open('tweets_DM.json', 'r')]

In [3]:
ms={}
# get ID and text
for item in data:
    ms[item['_source']['tweet']['tweet_id']]=item['_source']['tweet']['text']

In [4]:
# into Dataframe
df_ms = pd.DataFrame.from_dict(ms, orient='index',columns=['text'])
df_ms.index.names = ['tweet_id']
df_ms = df_ms.reset_index()

In [5]:
# merge
df = pd.merge(df_ms,df_di)

In [6]:
# training data & test data, labels for training data
train = df[df['identification']=='train'].drop(['identification'],axis=1)
test = df[df['identification']=='test'].drop(['identification'],axis=1)
train = pd.merge(train,df_em)

# Sampling (30000 samples for each type of emotion)

In [51]:
joy=train[train['emotion']=='joy'].sample(30000)
anticipation=train[train['emotion']=='anticipation'].sample(30000)
trust=train[train['emotion']=='trust'].sample(30000)
sadness=train[train['emotion']=='sadness'].sample(30000)
disgust=train[train['emotion']=='disgust'].sample(30000)
fear=train[train['emotion']=='fear'].sample(30000)
surprise=train[train['emotion']=='surprise'].sample(30000)
anger=train[train['emotion']=='anger'].sample(30000)
new_train=pd.concat([joy,anticipation,trust,sadness,disgust,fear,surprise,anger])

# CountVectorizer

In [59]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer

#tokenize, remove symbols and numbers
token = RegexpTokenizer(r'[a-zA-Z0-9]+')
cv = CountVectorizer(lowercase=True,stop_words='english',ngram_range = (1,3),tokenizer = token.tokenize)
counts= cv.fit_transform(train['text'])

In [60]:
from sklearn.model_selection import train_test_split
# training data: test data= 9:1
X_train, X_test, y_train, y_test = train_test_split(counts,train['emotion'], test_size=0.1, random_state=1)

In [61]:
# Naive Bayes

from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
clf = MultinomialNB().fit(X_train,y_train)
predicted = clf.predict(X_test)
print("MultinomialNB Accuracy:",metrics.accuracy_score(y_test, predicted))

MultinomialNB Accuracy: 0.5522441380352714


In [55]:
counts2 = cv.transform(test['text'])
predicted_test = clf.predict(counts2)

In [56]:
import csv
# output csv
with open('output.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['id','emotion'])
    for tweet_id,emo in zip(test['tweet_id'],predicted_test):
        writer.writerow([tweet_id,emo])

# TFIDF

In [7]:
# TFIDF

from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer()
text_tf = tf.fit_transform(train['text'])

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(text_tf,train['emotion'], test_size=0.1, random_state=123)

In [11]:
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
clf2 = MultinomialNB().fit(X_train,y_train)
predicted = clf2.predict(X_test)
print("MultinomialNB Accuracy:",metrics.accuracy_score(y_test, predicted))

MultinomialNB Accuracy: 0.46440226165694537


In [14]:
import csv
counts2=tf.transform(test['text'])
predicted_test = clf2.predict(counts2)
with open('output.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['id','emotion'])
    for tweet_id,emo in zip(test['tweet_id'],predicted_test):
        writer.writerow([tweet_id,emo])

# SGDClassifier

In [62]:
from sklearn.linear_model import SGDClassifier
from sklearn import metrics
sgd_clf = SGDClassifier(random_state=42).fit(X_train, y_train)
predicted = sgd_clf.predict(X_test)
print("SGDClassifier Accuracy:",metrics.accuracy_score(y_test, predicted))

SGDClassifier Accuracy: 0.5703126610193945


In [63]:
import csv
# output csv
counts2 = cv.transform(test['text'])
predicted_test = sgd_clf.predict(counts2)
with open('output.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['id','emotion'])
    for tweet_id,emo in zip(test['tweet_id'],predicted_test):
        writer.writerow([tweet_id,emo])

# SVM

In [ ]:
from sklearn import svm
SVM = svm.SVC(kernel='linear')
SVM.fit(X_train,y_train)
predicted= SVM.predict(X_test)
print("SVM Accuracy:",metrics.accuracy_score(y_test, predicted))

# Decision Tree

In [13]:
from sklearn import tree
clf3 = tree.DecisionTreeClassifier()
clf3.fit(X_train,y_train)
predicted= clf3.predict(X_test)
print("decision tree Accuracy:",metrics.accuracy_score(y_test, predicted))

decision tree Accuracy: 0.1875
